In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
stepdata = pd.read_csv('stepdata.csv')
stepdata.head()


In [ ]:
plt.figure(figsize = (15,15))

plt.subplot(3, 1, 1)
plt.plot(stepdata['Time (s)'],stepdata['Linear Acceleration x (m/s^2)']) 
plt.grid()
plt.title('Acceleration x')
plt.xlabel('Time')
plt.ylabel('a_x')
plt.axis([-1,53,-10,10])

plt.subplot(3, 1, 2)
plt.plot(stepdata['Time (s)'],stepdata['Linear Acceleration y (m/s^2)']) 
plt.grid()
plt.title('Acceleration y')
plt.xlabel('Time')
plt.ylabel('a_y')
plt.axis([-1,53,-10,10])

plt.subplot(3, 1, 3)
plt.plot(stepdata['Time (s)'],stepdata['Linear Acceleration z (m/s^2)']) 
plt.grid()
plt.title('Acceleration z')
plt.xlabel('Time')
plt.ylabel('a_z')
plt.axis([-1,53,-10,10])

In [ ]:
from scipy.signal import butter, filtfilt

In [ ]:
def butter_lowpass_filter(data, cutoff, fs, nyq, order):
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y

def butter_highpass_filter(data, cutoff, fs, nyq, order):
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    y = filtfilt(b, a, data)
    return y


In [ ]:
noisy_data = stepdata['Linear Acceleration z (m/s^2)']
time = stepdata['Time (s)']

dt = (time[len(time)-1] - time[1])/len(time)
cutoff_H = 1/0.5
cutoff_L = 1/5
fs = 1/dt
nyq = 0.5 * fs
order = 1

lowpassed_data = butter_lowpass_filter(noisy_data, cutoff_L, fs, nyq, order)
highpassed_data = butter_highpass_filter(noisy_data, cutoff_H, fs, nyq, order)
clean_data = noisy_data - lowpassed_data - highpassed_data

In [ ]:
steps = 0
for i in range(clean_data.shape[0]-1):
    if clean_data[i]/clean_data[i+1] < 0:
        steps = steps + 0.5

In [ ]:
step_len = 0.415 * 174
dist = round(((steps * step_len) / 100), 2)
print(f'Steps: {steps}')
print(f'Distance: {dist}m')

In [ ]:
plt.figure(figsize = (15,15))

plt.subplot(2, 1, 1)
plt.plot(stepdata['Time (s)'], highpassed_data) 
plt.grid()
plt.title('Highpass')
plt.xlabel('Time')
plt.axis([-1,53,-8,8])

plt.subplot(2, 1, 2)
plt.plot(stepdata['Time (s)'], lowpassed_data) 
plt.grid()
plt.title('Lowpass')
plt.xlabel('Time')
plt.axis([-1,53,-1,1])

plt.show()

In [ ]:
plt.figure(figsize = (15,15))

plt.subplot(2, 1, 1)
plt.plot(stepdata['Time (s)'], noisy_data) 
plt.grid()
plt.title('Original signal')
plt.xlabel('Time')
plt.axis([-1,53,-8,8])

plt.subplot(2, 1, 2)
plt.plot(stepdata['Time (s)'], clean_data) 
plt.grid()
plt.title('Filtered signal')
plt.xlabel('Time')
plt.axis([-1,53,-8,8])